In [ ]:
import tifffile as tiff
from PIL import Image
import sys
import re
import gc
import time
import os
import zipfile
import cv2

import pandas as pd
from glob import glob
from tqdm.notebook import tqdm
import torch
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
from PIL import Image
import matplotlib.pyplot as plt
import tifffile as tiff
from google.colab.patches import cv2_imshow

from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
from skimage import io


In [ ]:

path = r'/content/drive/MyDrive/Break_his_stained/Cancer_validation/benign_stained' # Source Folder
dstpath = r'/content/drive/MyDrive/Breakhis_segmented/Cancer_validation/benign_stained_seg' # change URLs here.
target = tiff.imread('/content/drive/MyDrive/done.tiff')



try:
    os.makedirs(dstpath)
except:
    print ("Directory already exist, images will be written in asme folder")

# Folder won't used
files = os.listdir(path)

#for image in files:
for image in files:
 img = cv2.imread(os.path.join(path,image))
 b,g,r = cv2.split(img)
 rgb_img = cv2.merge([r,g,b])

 gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
 ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# noise removal
 kernel = np.ones((2,2),np.uint8)
#opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
 closing = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel, iterations = 2)

# sure background area
 sure_bg = cv2.dilate(closing,kernel,iterations=3)

# Finding sure foreground area
 dist_transform = cv2.distanceTransform(sure_bg,cv2.DIST_L2,3)

# Threshold
 ret, sure_fg = cv2.threshold(dist_transform,0.1*dist_transform.max(),255,0)

# Finding unknown region
 sure_fg = np.uint8(sure_fg)
 unknown = cv2.subtract(sure_bg,sure_fg)

# Marker labelling
 ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
 markers = markers+1

# Now, mark the region of unknown with zero
 markers[unknown==255] = 0

 markers = cv2.watershed(img,markers)
 img[markers == -1] = [255,0,0]
 cv2.imwrite(os.path.join(dstpath,image),img)